In [ ]:
import os
import cv2
import copy
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

import SimpleITK as stk
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from skimage import measure

In [ ]:
root = os.path.normpath('C:/Users/huixi/OneDrive - Durham University/Final Project/Dataset/LUNA-16/')
target_root = os.path.normpath('C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data')

In [ ]:
# Base path for the LUNA16 dataset
base_path = root

file_list = glob(f"{base_path}/subset/*.mhd")

# Print out the count of files for each subset
print(f"Files in subset Count:", len(file_list))

# Read the annotations file if it's located directly under the luna16 folder
annotations_df = pd.read_csv(f"{base_path}/annotations.csv")
print("Annotations DF Count:", len(annotations_df))
annotations_df.head()

In [ ]:
d = annotations_df['diameter_mm'].values
fig = plt.hist(d, bins=80)

In [ ]:
def get_filename(file_list, file):
    for f in file_list:
        if file in f:
            return f

In [ ]:
annotations_df["filename"] = annotations_df["seriesuid"].map(lambda file: get_filename(file_list, file))
annotations_df = annotations_df.dropna()
annotations_df = annotations_df[annotations_df['diameter_mm']>=3.0]
print(len(annotations_df))

In [ ]:
annotations_df.head()

In [ ]:
def load_mhd(file):
    mhdimage = stk.ReadImage(file)
    ct_scan = stk.GetArrayFromImage(mhdimage)
    origin = np.array(list(mhdimage.GetOrigin()))
    space = np.array(list(mhdimage.GetSpacing()))
    return ct_scan, origin, space

In [ ]:
def make_mask(img, center, diam):
    mask = np.zeros_like(img, dtype=np.uint8)
    mask = cv2.circle(mask, (abs(int(center[0])),abs(int(center[1]))),int(abs(diam//2)), 255, -1)
    return mask

In [ ]:
annotations_df_new = pd.read_csv(f"{base_path}/annotations.csv")
annotations_df_new

In [ ]:
n_neighbour = 4

In [ ]:
# Define the directories for saving output
nodule_mask_dir = os.path.join(target_root, "masks/")
lungs_roi_dir = os.path.join(target_root, "imgs/")

# Create the directories if they do not exist
os.makedirs(nodule_mask_dir, exist_ok=True)
os.makedirs(lungs_roi_dir, exist_ok=True)

In [ ]:
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))  # CLAHE(Contrast Limited Adaptive Histogram Equalization) filter for enhancing the contrast of an image

# Iterating over all the files in the subset
for i,file in tqdm(enumerate(np.unique(annotations_df['filename'].values))):
    annotations = annotations_df[annotations_df["filename"]==file]
    ct, origin, space = load_mhd(file)      # Loading the CT scan
    num_z, height, width = ct.shape
    ct_norm = cv2.normalize(ct, None, 0, 255, cv2.NORM_MINMAX)   # Normalizing the CT scan
    for idx, row in annotations.iterrows():
        node_x = int(row["coordX"])     # X coordinate of the nodule
        node_y = int(row["coordY"])     # Y coordinate of the nodule
        node_z = int(row["coordZ"])     # Z coordinate of the nodule
        diam = int(row["diameter_mm"])  # Diameter of the nodule

        center = np.array([node_x, node_y, node_z])   # nodule center (x,y,z)
        v_center = np.rint((center-origin)/space)   # nodule center in voxel space (still x,y,z ordering)
        v_diam = int(diam/space[0])+5       # Diameter of the nodule in voxel space

        img_norm_neighbours = []
        img_norm_improved_neighbours = []
        mask_neighbours = []
        img_norm = None
        img_norm_improved = None
        mask = None

        if 18<v_diam<22:              # If nodule diameter is of medium size the take two neighbour slides into consideration
            n_neighbour = 2

        min_i = max(0,(int(v_center[2])-n_neighbour))
        max_i = min((int(v_center[2])+n_neighbour),(num_z-1))
        n = max_i-min_i

        img_norm = ct_norm[int(v_center[2]),:,:]    # a slice of the CT scan containing the nodule
        img_norm = cv2.resize(img_norm, (512,512))  # Resizing the CT scan to 512x512
        img_norm_improved = clahe.apply(img_norm.astype(np.uint8))  # Applying CLAHE filter to the image
        mask = make_mask(img_norm, v_center, v_diam)    # Creating a mask of the nodule

        if v_diam>18:      # If the nodule is too big, we will also take neighboring slices
            for i in range(min_i, max_i+1):
                if i==int(v_center[2]):
                    continue

                im_n = ct_norm[i,:,:]
                im_n = cv2.resize(im_n, (512,512))
                im_n_improved = clahe.apply(im_n.astype(np.uint8))
                dia = int(2*abs(v_center[2]-i))    # Decrease mask diameter because nodule diameter decrease as we move away from its center
                msk = make_mask(im_n, v_center, v_diam-dia)
                img_norm_neighbours.append(im_n)
                img_norm_improved_neighbours.append(im_n_improved)
                mask_neighbours.append(msk)
            assert len(img_norm_neighbours)==len(img_norm_improved_neighbours)==len(mask_neighbours)==n

        # Calculating the threshold value for extracting the nodule mask using binary thresholding
        mask = cv2.bitwise_and(img_norm, img_norm, mask=cv2.dilate(mask,kernel=np.ones((5,5))))
        pts = mask[mask>0]
        kmeans2 = KMeans(n_clusters=2).fit(np.reshape(pts,(len(pts),1)))
        centroids2 = sorted(kmeans2.cluster_centers_.flatten())
        threshold2 = np.mean(centroids2)

        _, mask = cv2.threshold(mask, threshold2, 255, cv2.THRESH_BINARY)


        if v_diam>18:
            for i in range(n):
                mask_neighbours[i] = cv2.bitwise_and(img_norm_neighbours[i], img_norm_neighbours[i], mask=cv2.dilate(mask_neighbours[i],kernel=np.ones((5,5))))
                _, mask_neighbours[i] = cv2.threshold(mask_neighbours[i], threshold2, 255, cv2.THRESH_BINARY)


        # Calculating the threshold value to segment the lungs from CT scan slices using binary thresholding
        centeral_area = img_norm[100:400, 100:400]
        kmeans = KMeans(n_clusters=2).fit(np.reshape(centeral_area, [np.prod(centeral_area.shape), 1]))
        centroids = sorted(kmeans.cluster_centers_.flatten())
        threshold = np.mean(centroids)

        # Steps to segment the lungs from CT scan slices
        ret, lung_roi = cv2.threshold(img_norm, threshold, 255, cv2.THRESH_BINARY_INV)
        lung_roi = cv2.erode(lung_roi, kernel=np.ones([4,4]))
        lung_roi = cv2.dilate(lung_roi, kernel=np.ones([13,13]))
        lung_roi = cv2.erode(lung_roi, kernel=np.ones([8,8]))

        labels = measure.label(lung_roi)        # Labelling different regions in the image
        regions = measure.regionprops(labels)   # Extracting the properties of the regions
        good_labels = []
        for prop in regions:        # Filtering the regions that are not too close to the edges
            B = prop.bbox           # Regions that are too close to the edges are outside regions of lungs
            if B[2]-B[0] < 475 and B[3]-B[1] < 475 and B[0] > 40 and B[2] < 472:
                good_labels.append(prop.label)
        lung_roi_mask = np.zeros_like(labels)
        for N in good_labels:
            lung_roi_mask = lung_roi_mask + np.where(labels == N, 1, 0)

        # Steps to get proper segmentation of the lungs without noise and holes
        contours, hirearchy = cv2.findContours(lung_roi_mask,cv2.RETR_CCOMP,cv2.CHAIN_APPROX_SIMPLE)
        external_contours = np.zeros(lung_roi_mask.shape)
        for i in range(len(contours)):
            if hirearchy[0][i][3] == -1:  #External Contours
                area = cv2.contourArea(contours[i])
                if area>518.0:
                    cv2.drawContours(external_contours,contours,i,(1,1,1),-1)
        external_contours = cv2.dilate(external_contours, kernel=np.ones([4,4]))

        external_contours = cv2.bitwise_not(external_contours.astype(np.uint8))
        external_contours = cv2.erode(external_contours, kernel=np.ones((7,7)))
        external_contours = cv2.bitwise_not(external_contours)
        external_contours = cv2.dilate(external_contours, kernel=np.ones((12,12)))
        external_contours = cv2.erode(external_contours, kernel=np.ones((12,12)))

        img_norm_improved = img_norm_improved.astype(np.uint8)
        external_contours = external_contours.astype(np.uint8)      # Final segmentated lungs mask
        extracted_lungs = cv2.bitwise_and(img_norm_improved, img_norm_improved, mask=external_contours)

        mask = mask.astype(np.uint8)
        np.save(os.path.join(nodule_mask_dir, f"masks_{i}_{idx}.npy"), mask)
        np.save(os.path.join(lungs_roi_dir, f"lungs_{i}_{idx}.npy"), extracted_lungs)

        extracted_lungs_neighbours = [None]*n

        if v_diam>18:
            for i in range(n):
                img_norm_improved_neighbours[i] = img_norm_improved_neighbours[i].astype(np.uint8)
                extracted_lungs_neighbours[i] = cv2.bitwise_and(img_norm_improved_neighbours[i], img_norm_improved_neighbours[i], mask=external_contours)
                mask_neighbours[i] = mask_neighbours[i].astype(np.uint8)
                np.save(os.path.join(nodule_mask_dir, f"masks_{i}_{idx}_{i}.npy"), mask_neighbours[i])
                np.save(os.path.join(lungs_roi_dir, f"lungs_{i}_{idx}_{i}.npy"), extracted_lungs_neighbours[i])

In [ ]:
# Modelisation

After preparing my data, segmenting the lung areas, and saving the nodule masks along with the lung regions of interest (ROI), the next step is to build and train a deep learning model for segmentation or classification (cancer detection) based on our objectives. 

Here is how to proceed with modeling, training, and evaluating my model:

#  Data preparation.

##### Step 1: Load the images and masks.


The lung-extracted images (lungs_roi) and nodule masks (nodule_mask) are stored in .npy files. Now, we need to load them and associate them with the corresponding labels.

Visualizing both lung region images (ROI - Region of Interest) and nodule masks for multiple cases helps you understand where nodules are located and potentially whether these nodules can be indicative of cancer.

- The reason we use masks in this context is to specifically identify regions of interest (nodules) in lung images.
- A mask indicates the exact location of the nodule in the image, which is crucial for cancer analysis and detection. Deep learning models can use these masks to learn how to distinguish potential nodules from the rest of the lung tissue.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Exemple de chemin vers une image et son masque
image_path = 'C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data/imgs/lungs_5_0.npy'  # Mettez à jour avec votre chemin réel
mask_path = 'C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data/masks/masks_5_0.npy'  # Mettez à jour avec votre chemin réel

# Chargement de l'image et du masque
image = np.load(image_path)
mask = np.load(mask_path)

# Affichage de l'image
plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(image, cmap='gray')
plt.title('Image de région pulmonaire')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(mask, cmap='gray')
plt.title('Masque de nodule')
plt.axis('off')

plt.show()



This script loads and displays a specified number of images and masks.

- Visualizing these images helps you understand how nodules are represented in the data and how the masks identify these nodules.
- It is an essential step to check the quality of your preprocessing and to understand the data on which our model will be trained.

=> Lung segmentation (extraction of ROIs) and nodule masks are two key steps to isolate relevant areas for analysis and reduce noise and distractions in the images, allowing the model to focus only on the important parts.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

# Paths to the directories containing lung-extracted images and nodule masks
lungs_roi_dir = "C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data/imgs/"
nodule_mask_dir = "C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data/masks/"

# Obtention des chemins de fichiers
lungs_images_paths = glob(lungs_roi_dir + "*.npy")
nodule_masks_paths = glob(nodule_mask_dir + "*.npy")

# Get file paths
lungs_images_paths.sort()
nodule_masks_paths.sort()

# Ensure paths are sorted to match images with their masks
n_images_to_show = 5  # ou len(lungs_images_paths) pour afficher toutes les images

plt.figure(figsize=(10, 5 * n_images_to_show))

for i in range(min(n_images_to_show, len(lungs_images_paths))):
    # Define how many images you want to visualize
    image = np.load(lungs_images_paths[i])
    mask = np.load(nodule_masks_paths[i])
    
    plt.subplot(n_images_to_show, 2, 2*i+1)
    plt.imshow(image, cmap='gray')
    plt.title(f'Image {i+1}')
    plt.axis('off')
    
    plt.subplot(n_images_to_show, 2, 2*i+2)
    plt.imshow(mask, cmap='gray')
    plt.title(f'Mask {i+1}')
    plt.axis('off')

plt.tight_layout()
plt.show()


- Visualization of these images helps you understand how nodules are represented in the data and how the masks identify these nodules.

### Step 2: Preparation of X and y

- X: The loaded images (lungs_roi) serve as input data for our model.
- y: The nodule masks, after processing, act as labels indicating the presence or absence of nodules.
- Normalization: The images are normalized to have values between 0 and 1, which is a standard practice to enhance the training of deep learning models.
- Creation of y labels: We have defined y in a binary manner, where 1 indicates the presence of a nodule, and 0 indicates its absence. The logic behind this creation depends on the presence of non-zero pixels in the image, signifying the presence of nodules.

In [ ]:
import numpy as np
from glob import glob

# Paths to the directories containing lung-extracted images and nodule masks
lungs_roi_dir = "C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data/imgs/"
nodule_mask_dir = "C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data/masks/"

# Load images (X) and masks (y)
lungs_images_paths = glob(lungs_roi_dir + "*.npy")
nodule_masks_paths = glob(nodule_mask_dir + "*.npy")

# Ensure paths are sorted so that each image matches its mask
lungs_images_paths.sort()
nodule_masks_paths.sort()

X = np.array([np.load(path) for path in lungs_images_paths])
y = np.array([np.load(path) for path in nodule_masks_paths])

# Normalize the images
X = X / 255.0

# For labels, decide on the logic based on the presence or absence of a nodule:
# Here, a simplified example where y is binary: 1 if a nodule is present, 0 otherwise.
y = np.array([1 if np.any(mask) else 0 for mask in y])

### Step 3: Splitting into training, validation, and test sets



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2


In [ ]:
import os
import numpy as np

# For images
image_files = os.listdir('C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data/imgs/')
print("Number of images:", len(image_files))

# For masks
mask_files = os.listdir('C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data/masks/')
print("Number of masks:", len(mask_files))

# Check that each image has its corresponding mask
assert len(image_files) == len(mask_files), "The number of images and masks do not match."


# Augmentation

In [ ]:
import os

# Paths to directories where your image and mask files are located
images_dir = 'C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data/imgs/'
masks_dir = 'C:/Users/huixi/OneDrive - Durham University/Final Project/Main_Code_Paper/0-Code_Data/Data/masks/'

# List all files in the directories
image_files = os.listdir(images_dir)
mask_files = os.listdir(masks_dir)

# Ensure that for each image, there is a corresponding mask
for image_file in image_files:
    # Remove the extension to get the base identifier
    base_id = image_file.split('.')[0]
    # Replace 'lungs' with 'mask' in the base identifier to match the naming logic of mask files
    corresponding_mask = base_id.replace('lungs', 'masks') + '.npy'
    if corresponding_mask not in mask_files:
        print(f"Corresponding mask not found for the image: {image_file}")
